In [8]:
import numpy as np
import nltk
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel, LsiModel
from gensim import corpora
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

categories = ['rec.autos', 'comp.graphics', 'sci.space']
n = len(categories)
newsgroup = fetch_20newsgroups(subset='all', categories=categories, shuffle=True, remove=('headers', 'footers', 'quotes'))
print(newsgroup.filenames.shape)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(2950,)


In [13]:
tokenized_documents = [simple_preprocess(text) for text in newsgroup.data]
dictionary = corpora.Dictionary(tokenized_documents)
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]
modelTf = TfidfModel(bow_corpus)
tf_corpus = [modelTf[corpus_item] for corpus_item in bow_corpus]

In [14]:
lsi_model = LsiModel(tf_corpus,id2word=dictionary, num_topics=10)
vectorized_corpus = lsi_model[tf_corpus]

In [15]:
vectorized_corpus_new = []
for i in range(len(vectorized_corpus)):
    curr = []
    for j in range(len(vectorized_corpus[i])):
        curr.append(vectorized_corpus[i][j][1])
    curr = np.array(curr)
    vectorized_corpus_new.append(curr)
print(vectorized_corpus_new[0:3])
print(len(vectorized_corpus_new))
X_train, X_test, y_train, y_test = train_test_split(vectorized_corpus_new, newsgroup.target, test_size=0.33)
for i in range(len(X_train)-1,0,-1):
    if len(X_train[i])!=10:
        X_train.pop(i)
        y_train = np.delete(y_train, i, 0)
for i in range(len(X_test)-1,0,-1):
    if len(X_test[i])!=10:
        X_test.pop(i)
        y_test = np.delete(y_test, i, 0)

[array([ 0.08855883, -0.02869108,  0.01257973, -0.01696853, -0.01104496,
       -0.02606457, -0.03611559, -0.00641227, -0.03173769,  0.00714616]), array([ 0.22121344, -0.09886153, -0.06938536,  0.01602977,  0.13628215,
        0.10773511,  0.11459749, -0.01531818,  0.0252623 ,  0.01510546]), array([ 0.13276861, -0.05703473, -0.04403674,  0.02956769,  0.04425935,
        0.00974707, -0.0017282 , -0.00063986, -0.03447497, -0.00461112])]
2950


In [16]:
def train(X_train, X_test, y_train, y_test):
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return f1_score(y_test, y_pred, average='weighted')

In [17]:
print(len(X_train)==len(y_train) and len(X_test)==len(y_test))
print(train(X_train, X_test, y_train, y_test))

True
0.8110685961484335
